# Construction de quelques requêtes SQL

## Principe de fonctionnement

Les requêtes SQL seront ici effectuées dans une base de données SQLite,
accessible depuis Python. Le code que vous écrivez est donc du code Python,
qui effectue la requête dans la base de données.

Cette base de données est contenue dans le fichier `films.db`. Le module
`requestdb.py` a été écrit pour faciliter l'accès à la bse, et l'affichage des
tables. Vous pouvez tout à fait consulter ce fichier et vous en resservir avec vos élèves.


## Exemples d'utilisation

In [1]:
# Un peu de style :
from IPython.display import HTML
HTML("data/style.css")

In [2]:
# On commence par importer le module
from requestdb import RequestDB
# Puis on crée l'objet <db> qui en lui indiquant 
# le chemin vers la base de données (ça fonctionne
# avec n'importe quelle base sqlite)
db = RequestDB("data/films.db")

requestdb v (1, 1)


On peut à présent avoir des informations sur les tables contenues dans la
base:

In [ ]:
db.infos()

On voit par exemple que la table `personne` contient 5 champs nommés
`id`, `nom`, `pays`, `naissance` et `sexe`.

Pour écrire une requête, rien de plus simple. Voici par exemple comment lister
tous les pays (une fois chacun) dont ont connaît au moins une personne
de sexe féminin (donc une réalisatrice ou une actrice)

In [ ]:
db.query("SELECT DISTINCT pays FROM personne WHERE sexe='F' LIMIT 10")

Les difficultés supplémentaires, par rapport à de simples requêtes SQL 
sont de s'emmêler éventuellement les pinceaux avec les guillemets.
Il est conseillé de mettre la requête entre `"`, et 
de n'utiliser que des guillemets simples `'` dans la requête elle-même.

Pour plus de lisibilité, on peut écrire les requêtes sur plusieurs lignes
avec des `"""` ainsi (conseillé pour les requêtes longues, vous gagnerez en efficacité).

Enfin, ne mettez pas de `;` à la fin de la ligne `db.query`, qui doit toujours être la dernière ligne de la cellule (sinon, le tableau de résultat ne s'affichera pas).

In [4]:
r = """\
SELECT DISTINCT pays 
FROM personne 
WHERE 
   sexe='F' AND
   pays LIKE 'A%'
"""
db.query(r)

pays
Australia
Afrique du Sud
Austria
Algeria
Algérie
Argentina.


Voici un exemple avec une apostrophe dans la requête (il suffit de la 
doubler pour qu'elle ne soit pas confondue avec un guillemet, c'est une règle
SQL) :

In [5]:
r = """\
SELECT * FROM film WHERE titre LIKE 'L''A%'
"""
db.query(r)

id,titre,annee,nbSpectateurs,idRealisateur,idGenre
63,L'Armée des 12 singes,1995,21105000,280,878
761,L'Aile ou la Cuisse,1976,None,11201,35
944,L'Arme fatale 4,1998,35680575,7187,28
1555,L'Auberge espagnole,2002,487224,17520,18
9944,L'affaire Pélican,1993,24408507,6349,18
12531,L'Appartement,1996,None,72667,9648
15383,L'Armée des ombres,1969,92720,3831,10752
18487,L'Attaque du métro 123,2009,18770765,893,53
30432,L'Armée du crime,2009,None,5069,10752
34280,L'Aventure c'est l'aventure,1972,None,16862,12


Attention à ne pas produire de tables résultat trop grandes (le navigateur pourrait ne pas apprécier...).
Il y a un garde-fou. Si le résultat contient plus de 100 lignes, il ne s'affichera pas.
Vous pouvez toutefois forcer l'affichage avec la syntaxe :

`dq.query(<requete>, force=True)`

**Attention à ne pas écrire de requêtes trop gourmandes en mémoire**. Si cela se produit, repérez que le notebook est en cours de calcul (astérisque dans la marge gauche, voir ci-dessous). Il faut alors stopper le calcul en allant dans le menu *Kernel/Interrupt*, ou cliquer sur le bouton stop ■. Si vous ne le faites pas, toutes les cellules que vous exécuterez pas la suite seront à leur tour «bloquées».

![grosserequete_nb.png](attachment:grosserequete_nb.png)

## Requêtes corrigées

Par rapport à une BDD mysql interrogée directement, ce système avec un notebook ne permet que de faire des requêtes de type `SELECT` (pas de `DESCRIBE` ou `SHOW`). Pour avoir des informations sur les tables, vous devez écire ici `db.info()` (voir plus haut dans le notebook).

#### 10 noms de personnes

In [6]:
r = """\
SELECT nom FROM personne LIMIT 10
"""
db.query(r)

nom
George Lucas
Mark Hamill
Harrison Ford
Carrie Fisher
Anthony Daniels
Andrew Stanton
Lee Unkrich
Bob Peterson
Albert Brooks
Brad Garrett


#### Noms et dates de naissance des 10 premières personnes dont la date de naissance est connue ('IS NOT NULL')

Donner la liste dans l'ordre chronologique (du plus vieux au plus jeune).

In [13]:
# ASC est optionnel
r = """\
SELECT nom, naissance FROM personne 
  WHERE naissance IS NOT NULL 
  ORDER BY naissance ASC LIMIT 10
"""
db.query(r)

nom,naissance
Phyllis Allen,1861-11-25
Henry Bergman,1868-02-22
John Rand,1871-11-19
Gabrielle Fontan,1873-04-16
Leo White,1873-11-10
Kitty Bradbury,1875-05-17
Mack Swain,1876-02-16
Steve Murphy,1876-12-25
Franklyn Farnum,1878-06-05
Eric Campbell,1879-04-26


#### Noms et dates de naissance des 10 premières personnes dont la date de naissance est connue ('IS NOT NULL')

Donner la liste dans l'ordre chronologique inverse (du plus jeune au plus vieux).

Les données viennent d'une BDD en ligne... il y a des erreurs manifestement :)

In [35]:
r = """\
SELECT nom, naissance 
FROM personne 
  WHERE naissance IS NOT NULL 
  ORDER BY naissance DESC LIMIT 10
"""
db.query(r)

nom,naissance
Christopher Thompson,2018-05-01
Camden Munson,2018-05-01
Thomas Arnold,2018-05-01
Madeleine McGraw,2008-12-22
Luke Roessler,2007-03-12
Noah Jupe,2005-02-25
Noah Schnapp,2004-10-03
Eden Estrella,2004-03-30
Wyatt Oleff,2003-07-13
Leilah de Meza,2003-05-09


#### Liste des titres des films commençant par la lettre 'M', par ordre alphabétique 

In [9]:
r = """\
SELECT titre FROM film WHERE titre LIKE 'M%' ORDER BY titre
"""
db.query(r)

titre
MR 73
Ma Loute
Ma famille t'adore déjà !
Ma vie de chat
Mad City
Madagascar
Madagascar 2
Madagascar 3 : Bons baisers d'Europe
Madame Doubtfire
Maigret tend un piège


#### Différents noms des acteurs français (l'attribut 'pays' doit valoir 'France')

Donner la liste dans l'ordre croissant des identifiants (attribut 'id'), limité à 30 résultats

Attention, il ne faut pas lister une personne qui aurait été simplement réalisatrice, sans jamais jouer dans un film (dans ce cas ce n'est pas un acteur...)

In [17]:
r = """\
SELECT DISTINCT nom FROM personne,joue WHERE pays='France' AND personne.id=joue.idActeur ORDER BY id ASC LIMIT 30
"""
db.query(r)

nom
Catherine Deneuve
Vernon Dobtcheff
Juliette Binoche
Hélène Vincent
Julie Delpy
Jean-Louis Trintignant
Jean-Pierre Léaud
Claire Maurier
Pierre Repp
Vincent Cassel


#### Nombre de réalisateurs par nationalité

Attention à bien comprendre l'intérêt de `DISTINCT` ici. 
Si on ne le met pas, un réalisateur ayant fait 2 films comptera deux fois. Au final, ce qu'on obtiendra, c'est le nombre de films, et non le nombre de réalisateurs... Notez comment on renomme une colonne, pour avoir un affichage plus propre. Notez aussi que la BDD contient quelques incohérences (U.S, U.S.A, United States... California ?...)

In [23]:
r = """\
SELECT pays, COUNT(DISTINCT personne.id) AS nombre
FROM personne, film
WHERE personne.id=film.idRealisateur
GROUP BY pays
"""
db.query(r)

pays,nombre
None,116
Algeria,1
Algérie,2
Argentina,1
Australia,5
Austria,2
Belgium,7
Bolivia,1
Brazil,2
California,1


#### Titres des films (et le nombre d'acteurs) dont le nombre d'acteurs est supérieur à 40 par ordre décroissant du nombre d'acteurs

Notez comment on renomme la colonne `COUNT(*)` et comment on utilise son nom dans les clauses `HAVING` et `ORDER BY`. 
On peut aussi ne pas la renommer et écrire `HAVING COUNT(*) > 40`, mais c'est moins lisible.

In [6]:
r = """\
SELECT titre, COUNT(*) as nbacteurs
FROM  joue, film
WHERE joue.idFilm = film.id
GROUP BY idFilm
HAVING nbacteurs > 40
ORDER BY nbacteurs DESC

"""
db.query(r)

titre,nbacteurs
And the Oscar Goes To...,120
Harry Potter et les Reliques de la mort : 1ère partie,75
Harry Potter et les Reliques de la mort : 2ème partie,73
Avengers: Endgame,69
The Dark Knight Rises,58
Le Seigneur des anneaux : Le Retour du roi,55
Le Loup de Wall Street,54
Harry Potter et l'Ordre du Phénix,52
Le Seigneur des anneaux : Les Deux Tours,51
Avengers : Infinity War,49


## Exercices

**C'est à vous maintenant !!!!**

Dans la suite, vous devez écrire la requête qui fournit la réponse à la
question rédigée. Chaque question indique bien la liste des colonnes qu'on souhaite voir. Il n'en faut ni plus, ni moins.

On utilise indifféremment dans la suite : ordre alphabétique, lexicographique, alphanumérique... Il s'agit dans tous les cas de l'ordre de SQL sur les chaînes de caractère (c'est l'ordre sur les codes ASCII des caractères)

Toutes les questions doivent être faites en une seule requête. Si on vous demande les films réalisés par Scorsese, par exemple, vous ne pouvez pas écrire une première requête pour relever le numéro `id` de Scorese, puis utiliser ce numéro dans une autre requête pour répondre à la question.

### 1. Requêtes simples

#### 1.1 Liste des titres des films commençant par la lettre `M` et se terminant par la lettre `r` ou `s`

In [19]:
r = """\
SELECT titre
FROM film
WHERE titre LIKE 'M%s' OR titre LIKE 'M%r'
"""
db.query(r)

titre
Mystic River
Moonraker
Madagascar
Manon des sources
Marrakech Express
"Musée haut, musée bas"
Meurs un autre jour
Millénium : Les hommes qui n'aimaient pas les femmes
Mon pire cauchemar
Mes héros


#### 1.2 Liste des titres des films sortis entre 2002 et 2004 (sur les 3 années)

In [21]:
r = """\
SELECT titre,annee
FROM film
WHERE annee BETWEEN 2002 AND 2004
"""
db.query(r)

titre,annee
Kill Bill : Volume 1,2003
Million Dollar Baby,2004
Hero,2002
Le Seigneur des anneaux : Les Deux Tours,2002
Le Seigneur des anneaux : Le Retour du roi,2003
Ocean's Twelve,2004
Minority Report,2002
Bruce tout-puissant,2003
Mystic River,2003
Monsieur Ibrahim et les fleurs du Coran,2003


#### 1.3 Liste des noms et dates de naissance des personnes connues de la base dont le prénom commence par 'Ro' et de nationalité Française (le pays vaut 'France'), par ordre alphabétique.

In [23]:
r = """\
SELECT nom, naissance
FROM personne
WHERE nom LIKE 'Ro%' AND pays='France'
ORDER BY nom
"""
db.query(r)

nom,naissance
Robert Berri,1912-12-17
Robert Dalban,1903-07-19
Robert Guédiguian,1953-12-03
Robert Hossein,1927-12-30
Robert Le Vigan,1900-01-05
Robert Manuel,1916-09-07
Robert Porte,1923-03-09
Robinson Stévenin,1981-03-01
Roger Carel,1927-08-14
Roger Dumas,1932-05-09


### 2. Jointures

#### 2.1 Liste des films sortis en 2006 en indiquant le titre, et le genre (en texte), par ordre alphabétique des titres.

In [26]:
r = """\
SELECT titre, nom AS genre
FROM film, genre
WHERE annee=2006 AND genre.id=film.idGenre
ORDER BY titre
"""
db.query(r)

titre,genre
Arthur et les Minimoys,Aventure
Black book,Drame
Blood Diamond,Drame
Borat : Leçons culturelles sur l'Amérique au profit de la glorieuse nation Kazakhstan,Comédie
Camping,Comédie
Cars : Quatre roues,Animation
Casino Royale,Aventure
Da Vinci Code,Thriller
Dikkenek,Comédie
Déjà vu,Action


#### 2.2 Liste des drames sortis strictement avant 1970 en donnant le titre et l'année de sortie...
...par ordre chronologique (premier critère), puis alphabétique des titres (second critère de tri, pour les année de sorties égales)

In [35]:
r = """\
SELECT titre, annee, nom AS genre
FROM film, genre
WHERE film.idgenre=genre.id AND genre.nom='Drame' AND film.annee<1970
ORDER BY annee, titre
"""
db.query(r)

titre,annee,genre
Les temps modernes,1936,Drame
La Grande Illusion,1937,Drame
La Bête humaine,1938,Drame
Le Silence de la mer,1949,Drame
Les Enfants terribles,1950,Drame
Le Plaisir,1952,Drame
Les feux de la rampe,1952,Drame
Bob le flambeur,1956,Drame
La Traversée de Paris,1956,Drame
Voici le temps des assassins...,1956,Drame


####  2.3 Liste des genres de films réalisés par Martin Scorsese (sans que le même genre ne s'affiche deux fois).

In [56]:
r = """\
SELECT DISTINCT genre.nom
FROM film, genre, personne
WHERE film.idGenre=genre.id AND film.idRealisateur=personne.id AND personne.nom LIKE '%Scorsese%'
"""
db.query(r)

nom
Drame
Aventure
Crime


#### 2.4 Liste alphabétique des acteurs du film _Le Convoyeur_

Essayez de faire les jointures avec `WHERE` et `JOIN`.

In [60]:
r = """\
SELECT nom
FROM film, joue, personne
WHERE personne.id=joue.idActeur AND film.id=joue.idFilm AND film.titre LIKE '%Convoyeur%'
ORDER BY titre
"""
db.query(r)

nom
François Berléand
Albert Dupontel
Aure Atika
Philippe Laudenbach
Julien Boisselier
Jean Dujardin
Claude Perron


#### 2.5 Liste des films dont un des acteurs n'avait pas plus de 12 ans à la sortie du film.

Chaque ligne contiendra le titre du film, le nom de l'acteur, la date de sortie du film, la date de naissance de l'acteur et son âge à la sortie du film.
Un film pourra sortir plusieurs fois si plusieurs acteurs sont concernés.
Pour extraire une année numérique à partie d'une date de naissance, vous pouvez utiliser :

- `YEAR(naissance)` avec MariaDB
- `CAST(strftime("%Y", naissance) AS INTEGER)` avec SQLite... (la simple soustraction `annee - naissance` fonctionne aussi ici).

Il y a des entrées erronées dans la base... vous risquez d'avoir des âges négatifs (à cause de dates de naissance fausses), mais ce n'est pas à vous de régler ce problème :)

In [65]:
r = """\
SELECT titre, nom, annee, naissance,annee-CAST(strftime("%Y", naissance) AS INTEGER) as age FROM film, personne, joue WHERE joue.idFilm=film.id AND joue.idActeur=personne.id AND annee-YEAR(naissance)<=12
"""
db.query(r)

OperationalError : no such function: YEAR
No results


### 3. Agrégations
Souvent, dans ces questions, il est utile de d'abord produire une liste de réponses, et après avoir vérifié que c'était *a priori* correct, d'ajouter les opérateurs d'agrégation dans la requête.

#### 3.1 Table contenant seulement l'année de sortie du plus ancien film, et l'année de sortie du film le plus récent.

In [67]:
r = """\
SELECT MIN(annee) AS mini, MAX(annee) AS maxi
FROM film
"""
db.query(r)

mini,maxi
1916,2020


#### 3.2 Moyenne du nombre de spectateurs  par film.

In [3]:
r = """\
SELECT AVG(nbSpectateurs) AS moyenne
FROM film
"""
db.query(r)

moyenne
26588669.345177665


#### 3.3 Table contenant, pour chaque genre, le nombre de films qui relèvent de ce genre, par ordre décroissant du nombre de films

In [7]:
r = """\
SELECT genre.nom, COUNT(*) AS nombre
FROM film,genre
WHERE genre.id=film.idGenre
GROUP BY genre.nom
ORDER BY nombre DESC
"""
db.query(r)

nom,nombre
Comédie,337
Drame,291
Action,140
Aventure,110
Crime,72
Thriller,40
Fantastique,32
Romance,29
Animation,16
Mystère,13


#### 3.4 Liste des 10 films comptant le plus d'acteurs...
... par ordre décroissant du nombre d'acteur, puis (2e critère) ordre alphabétique des titres

Attention, il pourrait y avoir des films différents avec le même titre !

In [10]:
r = """\
SELECT titre, COUNT(*) AS nombre_acteurs
FROM film, joue
WHERE joue.idFilm=film.id
GROUP BY film.id
ORDER BY nombre_acteurs DESC, titre ASC
LIMIT 10
"""
db.query(r)

titre,nombre_acteurs
And the Oscar Goes To...,120
Harry Potter et les Reliques de la mort : 1ère partie,75
Harry Potter et les Reliques de la mort : 2ème partie,73
Avengers: Endgame,69
The Dark Knight Rises,58
Le Seigneur des anneaux : Le Retour du roi,55
Le Loup de Wall Street,54
Harry Potter et l'Ordre du Phénix,52
Le Seigneur des anneaux : Les Deux Tours,51
Avengers : Infinity War,49


#### 3.5 Liste des films ayant le salaire moyen d'acteurs supérieur 2 millions...
par ordre décroissant du salaire moyen.

Note : Le champ salaire a été rempli aléatoirement (unité €, dollars ou millions de dollars, c'est selon...).


In [22]:
r = """\
SELECT film.id, titre, AVG(salaire) AS salaire_moyen
FROM film, joue
WHERE joue.idFilm=film.id 
GROUP BY film.id
HAVING salaire_moyen>2000000
ORDER BY salaire_moyen DESC
"""
db.query(r)

id,titre,salaire_moyen
299534,Avengers: Endgame,3902327.5362318843
299536,Avengers : Infinity War,3527093.8775510206
284053,Thor : Ragnarok,3120404.210526316
99861,Avengers : L'Ère d'Ultron,3084056.2162162163
141052,Justice League,2581400.0
278927,Le Livre de la jungle,2434600.0
337339,Fast & Furious 8,2269022.727272727
384018,Fast & Furious : Hobbs & Shaw,2149666.6666666665
209112,Batman v Superman : L’aube de la justice,2105196.9696969697


#### 3.6  Recette totale générée par des films de réalisateurs américains (vous utiliserez pays = 'USA')  sortis en 2010 ...
... en prenant un prix de l'entrée à 8 €

In [30]:
r = """\
SELECT 8*SUM(nbSpectateurs) AS recette_totale
FROM film, personne
WHERE personne.id=film.idRealisateur AND personne.pays='USA' AND film.annee=2010
"""
db.query(r)

recette_totale
2821683904


### Sous-requêtes

**Liste des films (avec leur année de sortie) sortis au pire 2 ans après le film le plus ancien de la base.**

Vous n'avez pas le droit de chercher cette date, puis de la mettre en dur dans la requête, il faut que votre requête fonctionne sans modification *quelles que soient les données présentes dans la base*.

In [34]:
r = """\
SELECT titre, annee
FROM film 
WHERE annee<3+(SELECT MIN(annee) FROM film)
"""
db.query(r)


titre,annee
Charlot s'évade,1917
Une vie de chien,1918
Charlot policeman,1917
L'Émigrant,1917
Charlot pompier,1916
Charlot rentre tard,1916
Charlot soldat,1918
